#  Analyze EGFD Accountability Data 

E.Quinn  7/28/2018

In [1]:
import re
import numpy as np
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import struct
from datetime import datetime
import datetime

In [2]:
pd.set_option('display.max_rows', 3000)

## Dictionary for looking up name by page

In [3]:
FF_name = {1: 'FF Andrade', 2: 'FF Archambault', 3: 'Lt Babcock', 4: 'Lt Bailey', 5: 'Lt Beaudreau', \
    6: 'FF Campbell', 7: 'FF Columbier', 8: 'Prob-FF Crute', 9: 'FF DeLuca', 10: 'FF Forte', \
    11: 'Lt Gardner', 12: 'Prob-FF Gorman', 13: 'Lt Grady', 14: 'Lt Greene', \
    15: 'Lt Hall', 16: 'FF Howard', 17: 'Lt Jones', 18: 'FF King', 19: 'FF Lang Jr.', 20: 'Prob-FF Lavallee', \
    21: 'FF Marsh', 22: 'Lt Matola, Jr.', 23: 'FF McKeon', 24: 'Capt Mears', 25: 'Lt Monaghan', \
    26: 'Capt Montville', 27: "FF O'Donnell", 28: 'Prob-FF Perry', 29: 'Lt Perry', 30: 'Prob-FF Preston', \
    31: 'Lt Purcell', 32: 'Lt Richardson', 33: 'FF Snowling', 34: 'FF Stabile', 35: 'FF Szerlag', 36: 'Lt Warner III'}                 

## Dictionary for looking up page by name

In [4]:
FF_ID = {'FF Andrade': 1,'FF Archambault': 2,'Lt Babcock': 3,'Lt Bailey': 4,'Lt Beaudreau': 5,'FF Campbell': 6, \
         'FF Columbier': 7,'Prob-FF Crute': 8,'FF DeLuca': 9,'FF Forte': 10,'Lt Gardner': 11,'Prob-FF Gorman': 12, \
         'Lt Grady': 13,'Lt Greene': 14,'Lt Hall': 15,'FF Howard': 16,'Lt Jones': 17,'FF King': 18,'FF Lang Jr.': 19, \
         'Prob-FF Lavallee': 20,'FF Marsh': 21,'Lt Matola, Jr.': 22,'FF McKeon': 23,'Capt Mears': 24,'Lt Monaghan': 25, \
         'Capt Montville': 26,"FF O'Donnell": 27,'Prob-FF Perry': 28,'Lt Perry': 29,'Prob-FF Preston': 30,  \
         'Lt Purcell': 31,'Lt Richardson': 32,'FF Snowling': 33,'FF Stabile': 34,'FF Szerlag': 35,'Lt Warner III': 36}                 


## Dictionary of short names

In [5]:
FF_short_name = {1: 'FF DA', 2: 'FF SA', 3: 'Lt SB', 4: 'Lt TB', 5: 'Lt RB', 6: 'FF AC', \
         7: 'FF MC', 8: 'PFF SC', 9: 'FF AD', 10: 'FF SF', 11: 'Lt RG', 12: 'PFF DG', 13: 'Lt RG', 14: 'Lt MG',  \
         15: 'Lt KH', 16: 'FF MH', 17: 'Lt MJ', 18: 'FF KK', 19: 'FF KL', 20: 'PFF DL', 21: 'FF WM', \
         22: 'Lt EM', 23: 'FF SM', 24: 'Cpt TM', 25: 'Lt MM', 26: 'Cpt KM', 27: 'FF PO', 28: 'PFF JP',  \
         29: 'Lt BP', 30: 'PFF RP', 31: 'Lt WP', 32: 'Lt JR', 33: 'FF GS', 34: 'FF BS', 35: 'FF JS', 36: 'Lt RW'}                 


## Dictionary of initials by Page Number

In [6]:
FF_initials = {1: 'DA', 2: 'SA', 3: 'SB', 4: 'TB', 5: 'RB', 6: 'AC', 7: 'MC', 8: 'SC', 9: 'AD', 10: 'SF', 11: 'RG',  \
         12: 'DG', 13: 'RG', 14: 'MG', 15: 'KH', 16: 'MH', 17: 'MJ', 18: 'KK', 19: 'KL', 20: 'DL', 21: 'WM', \
         22: 'EM', 23: 'SM', 24: 'TM', 25: 'MM', 26: 'KM', 27: 'PO', 28: 'JP', 29: 'WP', 30: 'RP', 31: 'WP', \
         32: 'JR', 33: 'GS', 34: 'BS', 35: 'JS', 36: 'RW'}                 


## Dictionary for looking up platoon by name

In [7]:
FF_platoon = {'FF Andrade': 'C','FF Archambault': 'A','Lt Babcock': 'D','Lt Bailey': 'A','Lt Beaudreau': 'B','FF Campbell': 'A', \
         'FF Columbier': 'B','Prob-FF Crute': 'B','FF DeLuca': 'A','FF Forte': 'C','Lt Gardner': 'B','Prob-FF Gorman': 'D', \
         'Lt Grady': 'B','Lt Greene': 'B','Lt Hall': 'D','FF Howard': 'B','Lt Jones': 'C','FF King': 'C','FF Lang Jr.': 'D', \
         'Prob-FF Lavallee': 'D','FF Marsh': 'D','Lt Matola, Jr.': 'D','FF McKeon': 'C','Capt Mears': 'C','Lt Monaghan': 'A', \
         'Capt Montville': 'A',"FF O'Donnell": 'A','Prob-FF Perry': 'B','Lt Perry': 'B','Prob-FF Preston': 'A',  \
         'Lt Purcell': 'C','Lt Richardson': 'A','FF Snowling': 'B','FF Stabile': 'D','FF Szerlag': 'C','Lt Warner III': 'D'}                 


## Dictionary for looking up identifier by name

In [8]:
FF_token = {'FF Andrade': 'CFF2','FF Archambault': 'AFF1','Lt Babcock': 'DOF2','Lt Bailey': 'AOF2', \
        'Lt Beaudreau': 'BOF3','FF Campbell': 'AFF4', \
        'FF Columbier': 'BFF1','Prob-FF Crute': 'BFF4','FF DeLuca': 'AFF5','FF Forte': 'CFF4', \
        'Lt Gardner': 'BOF2','Prob-FF Gorman': 'DFF5', \
        'Lt Grady': 'BOF4','Lt Greene': 'COF4','Lt Hall': 'DOF1','FF Howard': 'BFF3','Lt Jones': 'COF2', \
        'FF King': 'CFF1','FF Lang Jr.': 'DFF3', \
        'Prob-FF Lavallee': 'DFF4','FF Marsh': 'DFF1','Lt Matola, Jr.': 'DOF4','FF McKeon': 'CFF5', \
        'Capt Mears': 'COF1','Lt Monaghan': 'AOF4', \
        'Capt Montville': 'AOF3',"FF O'Donnell": 'AFF2','Prob-FF Perry': 'BFF5','Lt Perry': 'BOF1', \
        'Prob-FF Preston': 'AFF3', 'Lt Purcell': 'COF3','Lt Richardson': 'AOF1','FF Snowling': 'BFF2', \
        'FF Stabile': 'DFF2','FF Szerlag': 'CFF3','Lt Warner III': 'DOF3','FF Squillante':'EFF1'}                 


## Dictionary for looking up name by identifier

In [9]:
token_FF = {'CFF2':'FF Andrade','AFF1':'FF Archambault','DOF2':'Lt Babcock','AOF2':'Lt Bailey', \
        'BOF3':'Lt Beaudreau','AFF4':'FF Campbell', \
        'BFF1':'FF Columbier','BFF4':'Prob-FF Crute','AFF5':'FF DeLuca','CFF4':'FF Forte', \
        'BOF2':'Lt Gardner','DFF5':'Prob-FF Gorman', \
        'BOF4':'Lt Grady','COF4':'Lt Greene','DOF1':'Lt Hall','BFF3':'FF Howard','COF2':'Lt Jones', \
        'CFF1':'FF King','DFF3':'FF Lang Jr.', \
        'DFF4':'Prob-FF Lavallee','DFF1':'FF Marsh','DOF4':'Lt Matola, Jr.','CFF5':'FF McKeon', \
        'COF1':'Capt Mears','AOF4':'Lt Monaghan', \
        'AOF3':'Capt Montville','AFF2':"FF O'Donnell",'BFF5':'Prob-FF Perry','BOF1':'Lt Perry', \
        'AFF3':'Prob-FF Preston', 'COF3':'Lt Purcell','AOF1':'Lt Richardson','BFF2':'FF Snowling', \
        'DFF2':'FF Stabile','CFF3':'FF Szerlag','DOF3':'Lt Warner III','EFF1':'FF Squillante'}                 


## Dictionary of 8-day cycle start times by platoon

In [10]:
base_date = {}
base_date['A'] = datetime.datetime(2016,12,26,7,0,0)   #first cycle start 1/1/2017 07:00 or earlier for this platoon
base_date['B'] = datetime.datetime(2016,12,28,7,0,0)
base_date['C'] = datetime.datetime(2016,12,30,7,0,0)
base_date['D'] = datetime.datetime(2017,1,1,7,0,0)

cycle_start = {}                                       # dictionary for cycle start by platoon

max_datetime = datetime.datetime(2018,7,1,0,0,0)

for shift_id in base_date.keys():                      # loop through earliest cycle start for each platoon
    cycle_start[shift_id] = {}                         # create dictionary for platoon
    ss = base_date[shift_id]                           # start with earliest cycle start time
    while (ss < max_datetime):                         # loop until cycle start is past max_datetime
        cycle_start[shift_id][ss] = {}                 # add dictionary for cycle start
        ss += datetime.timedelta(days=8)               # increment cycle start by 8 days

cycle_start

print('Number of 8-Day Cycles')
print('A ' + str(len(cycle_start['A'])))
print('B ' + str(len(cycle_start['B'])))
print('C ' + str(len(cycle_start['C'])))
print('D ' + str(len(cycle_start['D'])))

Number of 8-Day Cycles
A 69
B 69
C 69
D 69


In [11]:
base_date

{'A': datetime.datetime(2016, 12, 26, 7, 0),
 'B': datetime.datetime(2016, 12, 28, 7, 0),
 'C': datetime.datetime(2016, 12, 30, 7, 0),
 'D': datetime.datetime(2017, 1, 1, 7, 0)}

## Build list of shifts for each FF

In [12]:
shifts = {}       

for platoon in ['A','B','C','D']:
    for ffn in ['OF1','OF2','OF3','OF4','FF1','FF2','FF3','FF4','FF5']:
        token = platoon+ ffn
        shifts[token] = {}
        shifts[token]['name'] = token_FF[token]
        shifts[token]['shifts'] = {}
        for cycle in cycle_start[platoon]:
            shift = cycle
            shifts[token]['shifts'][shift] = {'type':'scheduled'}
            shift += datetime.timedelta(days=1)   # increment cycle start by 10 hours + 1 day
            shifts[token]['shifts'][shift] = {'type':'scheduled'}
            shift += datetime.timedelta(days=1)
            shift += datetime.timedelta(hours=10)   # increment cycle start by 10 hours + 1 day
            shifts[token]['shifts'][shift] = {'type':'scheduled'}
            shift += datetime.timedelta(days=1)   # increment cycle start by 1 day
            shifts[token]['shifts'][shift] = {'type':'scheduled'}
        

## Lookup dictionary to get accountability file names by FF

These are the .csv files extracted from Eric's spreadsheet of accountability data

In [13]:
logs = {'FF Andrade': 'FF_Andrade.csv', \
                       'FF Archambault': 'FF_Archambault.csv', \
                       'Lt Babcock': 'Lt_Babcock.csv', \
                       'Lt Bailey': 'Lt_Bailey.csv', \
                       'Lt Beaudreau': 'Lt_Beaudreau.csv', \
                       'FF Campbell': 'FF_Campbell.csv', \
                       'FF Columbier': 'FF_Columbier.csv', \
                       'Prob-FF Crute': 'Prob-FF_Crute.csv', \
                       'FF DeLuca': 'FF_DeLuca.csv', \
                       'FF Forte': 'FF_Forte.csv', \
                       'Lt Gardner': 'Lt_Gardner.csv', \
                       'Prob-FF Gorman': 'Prob-FF_Gorman.csv', \
                       'Lt Grady': 'Lt_Grady.csv', \
                       'Lt Greene': 'Lt_Greene.csv', \
                       'Lt Hall': 'FF_Hall.csv', \
                       'FF Howard': 'FF_Howard.csv', \
                       'Lt Jones': 'Lt_Jones.csv', \
                       'FF King': 'FF_King.csv',
                       'FF Lang Jr.': 'FF_Lang.csv', \
                       'Prob-FF Lavallee': 'Prob-FF_Lavallee.csv', \
                       'FF Marsh': 'FF_Marsh.csv', \
                       'Lt Matola, Jr.': 'Lt_Matola.csv', \
                       'FF McKeon': 'FF_McKeon.csv', \
                       'Capt Mears': 'Capt_Mears.csv', \
                       'Lt Monaghan': 'Lt_Monaghan.csv', \
                       'Capt Montville': 'Capt_Montville.csv', \
                       "FF O'Donnell": 'FF_ODonnell.csv', \
                       'Prob-FF Perry': 'Prob-FF_Perry.csv', \
                       'Lt Perry': 'Lt_Perry.csv', \
                       'Prob-FF Preston': 'Prob-FF_Preston.csv',  \
                       'Lt Purcell': 'Lt_Purcell.csv', \
                       'Lt Richardson': 'Lt_Richardson.csv', \
                       'FF Snowling': 'FF_Snowling.csv', \
                       'FF Stabile': 'FF_Stabile.csv', \
                       'FF Szerlag': 'FF_Szerlag.csv', \
                       'Lt Warner III': 'Lt_Warner.csv'}                 

## Read the accountability logs and add the data to the dictionaries

There is one .csv file containing the accountability data for each firefighter

Data elements are added to the dictionary for that firefighter and shift

In [14]:
def findOccurrences(s, ch):
    return [i for i, letter in enumerate(s) if letter == ch]

In [15]:
def clean_reason(reason,for_ff):
    new_reason = ''
    if (for_ff == ''):
        new_reason += reason
    else:
        if ((',' in reason) & ('-' in reason)):
            ix = findOccurrences(reason,'-')
            ix_last = ix[len(ix)-1]
            new_reason += FF_token[for_ff] + reason[ix_last:]

    return(new_reason)        

In [16]:
def read_acct(token):
    
    ff_name = token_FF[token]
    fname = '../' + logs[ff_name]                                   #get the filename for this FF
    acctdf = pd.read_csv(fname,parse_dates=[[0,1]],skiprows=2,header=None)   #read it
    acctdf.rename(columns={'0_1': 'shift',2:'type',3:'hours',4:'rank',5:'reason'},inplace=True)
    acctdf['FF_name'] = ff_name       #create a column for this FFs name
    
    for index, row in acctdf.iterrows():    #loop through row by row and add data to dictionary
        ff_name = row['FF_name']             #shifts dictionary key is FF name
        shift = row['shift'].to_pydatetime()
        dtv = datetime.datetime(shift.year,shift.month,shift.day,shift.hour,0,0)
        if dtv not in shifts[token]['shifts']:
            shifts[token]['shifts'][dtv] = {}
        
        try:                                      #create dictionary for accountability data
            sdict = shifts[token]['shifts'][dtv]['acct']
        except KeyError:                          #if this shift was not found add it
            shifts[token]['shifts'][dtv]['acct'] = {}
            sdict = shifts[token]['shifts'][dtv]['acct']
                                                  #fill in the accountability data elements
        typ = row['type']
        sdict['type'] = row['type']
        sdict['hours'] = float(row['hours'])
        sdict['rank'] = row['rank']
        sdict['reason'] = row['reason']
        rstr = row['reason']                      #determine whose shift was being covered
        for_ff = ''                               #search for specific name in reason string
        if isinstance(rstr, str):
            if ('Andrade' in rstr):  for_ff = 'FF Andrade'
            if ('Archambault' in rstr):  for_ff = 'FF Archambault'
            if ('Babcock' in rstr):  for_ff = 'Lt Babcock'
            if ('Bailey' in rstr): for_ff = 'Lt Bailey'
            if ('Beaudreau' in rstr): for_ff = 'Lt Beaudreau'
            if ('Campbell' in rstr): for_ff = 'FF Campbell'
            if ('Columbier' in rstr): for_ff = 'FF Columbier'
            if ('Crute' in rstr): for_ff = 'Prob-FF Crute'
            if ('DeLuca' in rstr): for_ff = 'FF DeLuca'
            if ('Forte' in rstr): for_ff = 'FF Forte'
            if ('Gardner' in rstr): for_ff = 'Lt Gardner'
            if ('Gorman' in rstr): for_ff = 'Prob-FF Gorman'
            if ('Grady' in rstr): for_ff = 'Lt Grady'
            if ('Greene' in rstr): for_ff = 'Lt Greene'
            if ('Hall' in rstr): for_ff = 'Lt Hall'
            if ('Howard' in rstr): for_ff = 'FF Howard'
            if ('Jones' in rstr): for_ff = 'Lt Jones'
            if ('King' in rstr): for_ff = 'FF King'
            if ('Lang' in rstr): for_ff = 'FF Lang Jr.'
            if ('Lavallee' in rstr): for_ff = 'Prob-FF Lavallee'
            if ('Marsh' in rstr): for_ff = 'FF Marsh'
            if ('Matola' in rstr): for_ff = 'Lt Matola, Jr.'
            if ('McKeon' in rstr): for_ff = 'FF McKeon'
            if ('Mears' in rstr): for_ff = 'Capt Mears'
            if ('Monaghan' in rstr): for_ff = 'Lt Monaghan'
            if ('Montville' in rstr): for_ff = 'Capt Montville'
            if ('Donnell' in rstr): for_ff = "FF O'Donnell"
            if ('FF Perry' in rstr): for_ff = 'Prob-FF Perry'
            if ('Lt Perry' in rstr): for_ff = 'Lt Perry'
            if ('Preston' in rstr): for_ff = 'Prob-FF Preston'
            if ('Purcell' in rstr): for_ff = 'Lt Purcell'
            if ('Richardson' in rstr): for_ff = 'Lt Richardson'
            if ('Snowling' in rstr): for_ff = 'FF Snowling'
            if ('Squillante' in rstr): for_ff = 'FF Squillante'
            if ('Stabile' in rstr): for_ff = 'FF Stabile'
            if ('Szerlag' in rstr): for_ff = 'FF Szerlag'
            if ('Warner' in rstr): for_ff = 'Lt Warner III'
            if (len(for_ff) > 0):
                shifts[token]['shifts'][dtv]['acct']['for_token'] = FF_token[for_ff]    #save FF being covered
            else:
                shifts[token]['shifts'][dtv]['acct']['for_token'] = ''  #no for_ff
            shifts[token]['shifts'][dtv]['acct']['clean_reason'] = clean_reason(rstr,for_ff)
    return()

In [17]:
for token in shifts.keys():
    read_acct(token)

In [19]:
#shifts['AFF1']

## Write web page displaying results

In [20]:
def cellcolor(token,ts,shifts):
    bgcolor = '#FFFFFF'
    try:
        typ = shifts[token]['shifts'][ts]['acct']['type']
        if (typ == 'VC'): bgcolor = '#C0C0C0'
        if (typ == 'IOD'): bgcolor = '#FF00FF'
        if (typ in ['PRS','BER']): bgcolor = '#FFFF00'
        if ('CS-' in typ): bgcolor = '#00FF00'
        if ('C-' in typ): bgcolor = '#00FF00'
    except KeyError:
        x=1
    return(bgcolor)

In [21]:
def line2(token,ts,shifts):
    l2str = '<br>'
    try:
        typ = shifts[token]['shifts'][ts]['acct']['type']
        tag = token + str(ts)
        if (typ in ['VC','IOD','SWAP','SWAPW','SL','PRS','BER']): 
            l2str += '<a href="#' + tag + '">' + typ + '</a>'
        if ('OT-' in typ): l2str += '<a href="#' + tag + '">' + typ + '</a>'
        if (('C-' in typ) | ('CS-' in typ)): l2str += '<a href="#' + tag + '">' + typ + '</a>'
    except KeyError:
        ;
    return(l2str)

In [22]:
def line3(token,ts,shifts):
    l3str = '<br>'
    
    try:
        if (shifts[token]['shifts'][ts]['acct']['for_token'] != ''):
            token2 = shifts[token]['shifts'][ts]['acct']['for_token']
            tag = token2 + str(ts)
            l3str += '<a href="#' + tag + '">' + token2 + '</a>'
        else:
            for token2 in shifts.keys():
                try:
                    if (shifts[token2]['shifts'][ts]['acct']['for_token'] == token):
                        tag = token2 + str(ts)
                        l3str += '<a href="#' + tag + '">' + token2 + '</a>'
                except KeyError:
                    ;

    except KeyError:
            ;
    return(l3str)

In [23]:
def shiftabl(token,year,month,hour,shifts,file):        
    ndays = {1:31,2:28,3:31,4:30,5:31,6:30,7:31,8:31,9:30,10:31,11:30,12:31}
    if (hour == 7): 
        full_shift=10
        rowstr = '<tr align="center"><td rowspan="2">' + str(month) + '/' + str(year) + '</td>'
        rowstr +='<td>07:00</td>'
    else: 
        full_shift=14
        rowstr = '<tr><td>17:00</td>'
    
    for d in range(1,1+ndays[month]):
        ts = datetime.datetime(year,month,d,hour,0,0)
        tag = token + str(ts)
        
        rowstr += '<td bgcolor="' + cellcolor(token,ts,shifts) + '">'
        
                                                        #see if we are scheduled this shift
        try:
            if (shifts[token]['shifts'][ts]['type']=='scheduled'):   #see if there is an entry for this timestamp
                rowstr += 'On'
        except KeyError:
            rowstr += '&nbsp;'
        rowstr += line2(token,ts,shifts)
        rowstr += line3(token,ts,shifts)
        rowstr += '</td>'
    file.write(rowstr + '</tr>\n')
    return()

In [24]:
file = open('../test_V2.html','w') 

file.write('<html><body>') 

for token in shifts.keys():
    ff_name = token_FF[token]
    file.write('<a name="' + ff_name + '">')
    file.write('<h2>' + token + ' ' + '</h2></a>\n')
    file.write('<table border="1">\n')
    
    file.write('<tr><td>Month</td><td>Shift</td>')
    for d in range(1,32):
        file.write('<td>' + str(d) + '</td>')
    file.write('</tr>\n')
    
    year = 2017
    month = 1
    
    for i in range(1,18):
            
        shiftabl(token,year,month,7,shifts,file)  
        shiftabl(token,year,month,17,shifts,file) 
        
        month += 1
        if (month > 12):
            year += 1
            month = 1
        
    file.write('</table>\n')
    
for token in shifts.keys():
    ff_name = token_FF[token]
    file.write('<a name="acct_' + token + '">\n')
    file.write('<h2>' + token + ' ' + '</h2></a>\n')
    file.write('<table border="1">\n')
    file.write('<tr><td>Shift</td><td>Type</td><td>Hours</td><td>Rank</td><td>Reason</td></tr>\n')

    for shift in shifts[token]['shifts'].keys():
        sdict = shifts[token]['shifts'][shift]
        try:
            entry_type = sdict['acct']['type']
            hours = sdict['acct']['hours']
            rank = sdict['acct']['rank']
            reason = sdict['acct']['clean_reason']
            tag = token + str(shift)
            file.write('<tr><td><a name="' + tag + '"></a>' + str(shift) + '</td>')
            file.write('<td>' + entry_type + '</td>')
            file.write('<td>' + str(hours) + '</td>')
            file.write('<td>' + rank + '</td>')
            file.write('<td>' + str(reason) + '</td></tr>\n')
        except KeyError:
            continue
    file.write('</table>\n')

file.close() 

In [25]:
for token in shifts.keys():
    print(token_FF[token])
    for shift in shifts[token]:
        try:
            typ = shifts[token]['shifts'][shift]['acct']['type']
            if typ in ['VC','SL','PER','IOD']:
                print(str(shift) + ' ' + typ)
                for ff_covering in shifts.keys():
                    try:
                        for_ff = shifts[ff_covering]['shifts'][shift]['acct']['for_ff']
                        if (for_ff == ff_name):
                            print(str(shift)+ ' ' + ff_covering + ' covering for: ' + for_ff )
                    except KeyError:
                        continue
        except KeyError:
            continue

Lt Richardson
Lt Bailey
Capt Montville
Lt Monaghan
FF Archambault
FF O'Donnell
Prob-FF Preston
FF Campbell
FF DeLuca
Lt Perry
Lt Gardner
Lt Beaudreau
Lt Grady
FF Columbier
FF Snowling
FF Howard
Prob-FF Crute
Prob-FF Perry
Capt Mears
Lt Jones
Lt Purcell
Lt Greene
FF King
FF Andrade
FF Szerlag
FF Forte
FF McKeon
Lt Hall
Lt Babcock
Lt Warner III
Lt Matola, Jr.
FF Marsh
FF Stabile
FF Lang Jr.
Prob-FF Lavallee
Prob-FF Gorman


In [27]:
counts = {}

for token in shifts:
    for shift in shifts[token]['shifts']:
        try:
            typ = shifts[token]['shifts'][shift]['acct']['type']
            if (typ == 'GT'): print(token_FF[token])
            if (typ in counts.keys()):
                counts[typ] += 1
            else:
                counts[typ] = 1
        except KeyError:
            ;

In [28]:
counts

{'SWAP': 445,
 'VC': 679,
 'SL': 432,
 'PRS': 78,
 'OT-VC': 618,
 'OT-PRS': 71,
 'OT-BERV': 23,
 'OT-SL': 365,
 'SWAPW': 441,
 'OT': 428,
 'DP': 48,
 'OT-ADM': 64,
 'OT-BARG': 15,
 'OT-IOD': 545,
 'OT-COMP': 62,
 'CS-APP': 7,
 'C-DD': 365,
 'CS-SO': 12,
 'C-ND': 325,
 'OT-EOD': 6,
 'IOD': 858,
 'CMP-U': 75,
 'ADMIN': 73,
 'CS-TRN': 14,
 'CMP-E': 76,
 'CT': 2,
 'C-FP': 8,
 'C-TRN': 5,
 'OFF': 1,
 'BER': 37,
 'ACTOT': 9,
 'SCH': 3,
 'C-DIVE': 14,
 'BARG': 16,
 'OT-VNCY': 64,
 'ACT': 68,
 'CS-EMS': 101,
 'UT': 2,
 'CS-FP': 2,
 'CS-FA': 122,
 'CS-SCBA': 7,
 'OT-SCH': 3,
 'C-BM': 15,
 'DFP': 8,
 'OT-EMERG': 2,
 'C-APP': 1,
 'EMER': 1}

## Check for situation of alleged 'gaming' OT and VAC

Scenario:  FF1 takes a vacation day, FF2 gets OT for covering FF1's vacation day
           Later in FF1's 8 day cycle, FF2 takes a vacation day and FF1 gets OT for covering it.
           
Algorithm: 

            For each FF, look at 8-day cycles in which he took a vacation day (or other day requiring coverage).  

            See if, in this 8-day cycle, this FF was paid for covering a day taken by the FF who covered his. 

In [ ]:
 match_count = 0

for ff_name in shifts.keys():                          # loop through FF list
    platoon = FF_platoon[ff_name]                      # get the platoon (A,B,C,D) for this FF
    ff_id = FF_ID[ff_name]                             # get page number
    print('Processing ' + ff_name + ' ' + str(ff_id) + ' ' + platoon)           # display name and id #
    for first_shift in cycle_start[platoon]:           # loop through shift start times that begin an 8-day cycle
        cycle_shifts = []                              # fill in the other shift start times for that cycle
        ts = first_shift
        for day in range(1,9):
            cycle_shifts.append(ts)
            ts += datetime.timedelta(hours=10)          
            cycle_shifts.append(ts)
            ts += datetime.timedelta(hours=14)
        for ts in cycle_shifts:                                      # loop through the shifts for this 8-day cycle
            try:
                stype = shifts[ff_name][ts]['acct']['type']          # get accountability log entry if there is one
                if (stype in ['VC','SL','PRS','ADM','BER','ACT']):   # see if coverage may be required
                    print(ff_name + ' ' + str(ts) + ' ' + stype)    
                    for ff_cover in shifts.keys():                   # see who covered this shift
                        if (ff_cover != 'ff_name'):
                            try:
                                if (shifts[ff_cover][ts]['acct']['for_id'] == ff_id):
                                    print('covered by: ' + ff_cover )
                                    cover_id = FF_ID[ff_cover]
                                    for cs in cycle_shifts:          # see if the ff who covered was himself covered
                                        try:                         # by the ff we are processing in this cycle
                                            if (shifts[ff_name][cs]['acct']['for_id'] == cover_id):
                                                typ = shifts[ff_name][cs]['acct']['type']
                                                print('match########## ' + str(cs) + ' ' + ff_name + ' for ' + ff_cover + ' ' + typ)
                                                if ('SWAP' not in typ): match_count += 1
                                        except KeyError:
                                            continue
                            except KeyError:
                                continue
            except KeyError:
                continue                                             #get here if no accountability log
                                
print(match_count)        

In [ ]:
def ptabl(platoon,ts,shifts,file):        
    file.write('<tr><td>' + str(ts) + '</td><td>' + platoon + '</td>')
    for i in range(1,5):
        ff_desig = 'OF' + platoon + str(i)
        ff_name = desig_FF[ff_desig]
        ff_cover = chk_cov(ff_name,ts)
        tag = ff_desig + str(ts)
        tds =('<td>')
        try:
            typ = shifts[ff_name]['shifts'][ts]['acct']['type']
            if (typ == 'IOD'):                      # set background purple for IOD
                tds = '<td bgcolor="#FF00FF">'
            if (typ == 'SL'):                       # set background gray for SL
                tds = '<td bgcolor="#C0C0C0">'
            if (typ == 'VC'):                       # set background light blue for VC
                tds = '<td bgcolor="#00FFFF">'
            if ((typ == 'PRS') | (typ == 'BER')):   # set background yellow for personal and bereavement 
                tds = '<td bgcolor="#FFFF00">'
            file.write(tds + '<a href="#' + tag + '">' + typ + '</a>')
        except KeyError:
            file.write(tds + 'On')
        if (len(ff_cover) > 2):
            cover_tag = FF_desig[ff_cover] + str(ts)
            file.write('<br><a href="#' + cover_tag + '">' + ff_cover + '</a>')
        file.write('</td>')
    for i in range(1,6):
        ff_desig = 'FF' + platoon + str(i)
        ff_name = desig_FF[ff_desig]
        ff_cover = chk_cov(ff_name,ts)
        tds =('<td>')
        try:
            typ = shifts[ff_name]['shifts'][ts]['acct']['type']
            tag = ff_desig + str(ts)
            if (typ == 'IOD'):                      # set background purple for IOD
                tds = '<td bgcolor="#FF00FF">'
            if (typ == 'SL'):                       # set background gray for SL
                tds = '<td bgcolor="#C0C0C0">'
            if (typ == 'VC'):                       # set background light blue for VC
                tds = '<td bgcolor="#00FFFF">'
            if ((typ == 'PRS') | (typ == 'BER')):   # set background yellow for personal and bereavement 
                tds = '<td bgcolor="#FFFF00">'
            file.write(tds + '<a href="#' + tag + '">' + typ + '</a>')
        except KeyError:
            if ((ff_name == 'Prob-FF Preston') & (ts < datetime.datetime(2017,6,1,0,0,0))):
                file.write(tds + 'NA')
            else:
                file.write(tds + 'On')
        if (len(ff_cover) > 2):
            cover_tag = FF_desig[ff_cover] + str(ts)
            file.write('<br><a href="#' + cover_tag + '">' + ff_cover + '</a>')
        file.write('</td>')
    file.write('</tr>\n')
    return()

In [ ]:
file = open('../platoons.html','w') 

file.write('<html><body>') 

for platoon in ['A','B','C','D']:
    file.write('<h2>Platoon: ' + platoon + '</h2></a>\n')
    file.write('<table border="1">\n')
    file.write('<tr><td>&nbsp;</td><td>&nbsp;</td>')
    for i in range(1,5):
        file.write('<td>OF' + platoon + str(i) + '</td>')
    for i in range(1,6):
        file.write('<td>FF' + platoon + str(i) + '</td>')
    file.write('</tr>\n')
    file.write('<tr><td>Shift</td><td>P</td>')
    for i in range(1,5):
        des = 'OF' + platoon + str(i)
        file.write('<td>'+ desig_FF[des] + '</td>')
    for i in range(1,6):
        des = 'FF' + platoon + str(i)
        file.write('<td>' + desig_FF[des]+ '</td>')
    file.write('</tr>\n')
        
    for ts in cycle_start[platoon]:
        if (ts < datetime.datetime(2018,5,21,1,0,0)):
            ts2 = ts
            ptabl(platoon,ts2,shifts,file)
            ts2 += datetime.timedelta(days=1)   #increment shift start by 1 day
            ptabl(platoon,ts2,shifts,file)
            ts2 += datetime.timedelta(days=1)   #increment shift start by 1 day
            ts2 += datetime.timedelta(hours=10)   #increment shift start 10 hours
            ptabl(platoon,ts2,shifts,file)
            ts2 += datetime.timedelta(days=1)   #increment shift start by 1 day
            ptabl(platoon,ts2,shifts,file)
    file.write('</table>\n')

acct_webpage(file,shifts)          #write webpage of accountability data with links

file.close() 